## This section is where I upload all of my tweets, clean and turn them into tsv files for BERT classifying.

In [12]:
import pandas as pd
import string
import nltk
wn = nltk.WordNetLemmatizer()

prefix = 'data/'

In [13]:
# my train data
df=pd.read_csv('all.csv')

In [14]:
new= pd.DataFrame(df[['text', 'label']])

In [15]:
# I know there's nan's in label section since I intentionally left it blank to try to achieve a more balance class
#while manually labeling the data
new.dropna(inplace=True)

In [16]:
#NOT FINAL DATA. HAVE MORE DATA TO PROCESS AND UPLOAD HERE
new.label.value_counts()

0.0    1850
1.0     348
Name: label, dtype: int64

In [17]:
#converting files that BERT understand. It requires an ID, label, alpha and text columns. 
train_df = pd.DataFrame({
    'id':range(len(new)),
    'label':new.iloc[:,-1],
    'alpha':['a']*new.shape[0],
    'text': new.iloc[:,-2].replace(r'\n', ' ', regex=True)
})

train_df.head()

,id,label,alpha,text
0,0,0.0,a,RT @HockeyinVegas: The @GoldenKnights' Marc-A...
1,1,0.0,a,"Pneumonia is dangerous guys it’s not the flu ,..."
2,2,1.0,a,I've lost 10 pounds in less than a week due to...
3,3,0.0,a,RT @greatestjubilee: there was actually a whol...
4,4,0.0,a,RT @hitchcockherd: You can find a family in ne...


In [195]:
df1=pd.read_csv('my_test_files.csv')

In [196]:
df1= pd.DataFrame(df1[['text', 'label']])
df1.dropna(inplace=True)

In [198]:
dev_df = pd.DataFrame({
    'id':range(len(df1)),
    'label':df1.iloc[:,-1],
    'alpha':['a']*df1.shape[0],
    'text': df1.iloc[:,-2].replace(r'\n', ' ', regex=True)
})

dev_df.head()

,id,label,alpha,text
0,0,0.0,a,RT @NWOMXNDX: Niggas posting chicken sandwiche...
1,1,0.0,a,RT @sallyKP: ALL THREE SIBLINGS HAD THE FLU VA...
2,2,0.0,a,RT @wirralct: #MyFluFact Flu... it's serious! ...
3,3,0.0,a,Going from the flu to racing NoHo. WHAT COULD ...
4,4,0.0,a,PLEASE SHARE UV Light 185nm-254nm Safe for Hu...


In [200]:
train_df.to_csv('train.tsv', sep='\t', index=False, header=False)
dev_df.to_csv('dev.tsv', sep='\t', index=False, header=False)


In [63]:
# our text cleaner
#removing URLs and string beginning with @ 
def clean_tweets(text):
    text= re.sub(r'@[A-Za-z0-9]+', '', text)
    text=re.sub(r'http\S+', '', text)
    text= re.sub(r'RT @[\w_]+:', '', text)
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    text= re.sub(r'\b\w{1,2}\b', '', text)
    text = re.split('\W+', text)
    text = [wn.lemmatize(word) for word in text]
    text= " ".join([word for word in text if not word.isdigit()])
    return text

In [65]:
train_df['clean']= train_df['text'].apply(lambda x: clean_tweets(x))
train_df

,id,label,alpha,text,clean
0,0,0.0,a,RT @HockeyinVegas: The @GoldenKnights' Marc-A...,The MarcAndre Fleury wa full participant prac...
1,1,0.0,a,"Pneumonia is dangerous guys it’s not the flu ,...",Pneumonia dangerous guy not the flu look after...
2,2,1.0,a,I've lost 10 pounds in less than a week due to...,Ive lost pound le than week due the amount tim...
3,3,0.0,a,RT @greatestjubilee: there was actually a whol...,there wa actually whole compilation way felt ...
4,4,0.0,a,RT @hitchcockherd: You can find a family in ne...,You can find family need whether it baby adop...
...,...,...,...,...,...
13536,2193,0.0,a,https://t.co/C1EngIvYkM,
13537,2194,1.0,a,Somehow within only being home for 3 days I’v...,Somehow within only being home for day come do...
13538,2195,0.0,a,"RT @martingeddes: ""And that’s a problem for ma...",And that problem for many people They would r...
13539,2196,0.0,a,RT @Juchtervbergen: #ai #android #app #flu #ge...,android app flu gear Weather Channel app us W...


In [70]:
# dropping unclean text then renaming the clean column 'text'
train_df.drop('text', axis=1, inplace=True)

In [75]:
train_df.rename(columns= {"clean": "text"}, inplace=True)

In [ ]:
# now doing the same for test file
dev_df['clean']= dev_df['text'].apply(lambda x: clean_tweets(x))
dev_df.drop('text', axis=1, inplace=True)
dev_df.rename(columns= {"clean": "text"}, inplace=True)

In [ ]:
#Saving our files TSV files
train_df.to_csv('data/train.tsv', sep='\t', index=False, header=False)
dev_df.to_csv('data/dev.tsv', sep='\t', index=False, header=False)